In [52]:
import pandas as pd

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [53]:
path_to_data_folder =  '/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/process_mining_data'

In [54]:
# extract the 1-10 lags

df_2017 = pd.read_csv(f"{path_to_data_folder}/BPI_2017_naive.csv")

df_2017['concept:name - lag_1'] = df_2017['concept:name'].shift(1)
df_2017['concept:name - lag_2'] = df_2017['concept:name'].shift(2)
df_2017['concept:name - lag_3'] = df_2017['concept:name'].shift(3)
df_2017['concept:name - lag_4'] = df_2017['concept:name'].shift(4)
df_2017['concept:name - lag_5'] = df_2017['concept:name'].shift(5)
df_2017['concept:name - lag_6'] = df_2017['concept:name'].shift(6)
df_2017['concept:name - lag_7'] = df_2017['concept:name'].shift(7)
df_2017['concept:name - lag_8'] = df_2017['concept:name'].shift(8)
df_2017['concept:name - lag_9'] = df_2017['concept:name'].shift(9)
df_2017['concept:name - lag_10'] = df_2017['concept:name'].shift(10)

df_2017 = df_2017.loc[10:]

# define target

df_2017['concept:name next'] = df_2017['concept:name'].shift(-1)

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2949031675.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2017 = pd.read_csv(f"{path_to_data_folder}/BPI_2017_naive.csv")


In [55]:
# test slices
df_2017 = df_2017.loc[:1000]

In [56]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,concept:name - lag_2,concept:name - lag_3,concept:name - lag_4,concept:name - lag_5,concept:name - lag_6,concept:name - lag_7,concept:name - lag_8,concept:name - lag_9,concept:name - lag_10,concept:name next
10,10,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584,Home improvement,New credit,...,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept
11,11,statechange,User_1,A_Concept,Application,ApplState_421204851,complete,2016-01-01 10:17:31.594,Home improvement,New credit,...,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application
12,12,Created,User_1,A_Create Application,Application,Application_428409768,complete,2016-01-01 11:19:38.177,Home improvement,New credit,...,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted
13,13,statechange,User_1,A_Submitted,Application,ApplState_105207793,complete,2016-01-01 11:19:38.235,Home improvement,New credit,...,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads
14,14,Created,User_1,W_Handle leads,Workflow,Workitem_1185806608,schedule,2016-01-01 11:19:38.914,Home improvement,New credit,...,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,996,statechange,User_1,A_Submitted,Application,ApplState_826687676,complete,2016-01-05 13:28:07.003,Home improvement,New credit,...,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads
997,997,Created,User_1,W_Handle leads,Workflow,Workitem_1696440271,schedule,2016-01-05 13:28:07.376,Home improvement,New credit,...,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads
998,998,Deleted,User_1,W_Handle leads,Workflow,Workitem_405441961,withdraw,2016-01-05 13:30:37.095,Home improvement,New credit,...,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application
999,999,Created,User_1,W_Complete application,Workflow,Workitem_473295245,schedule,2016-01-05 13:30:37.122,Home improvement,New credit,...,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,A_Concept


In [57]:
df_2017.columns

Index(['Unnamed: 0', 'Action', 'org:resource', 'concept:name', 'EventOrigin',
       'EventID', 'lifecycle:transition', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID',
       'timestamp_difference', 'event_seq', 'time_to_next_event',
       'time_to_next_event_seconds', 'predicted_time_to_next_event_seconds',
       'elapsed_time_from_start', 'predicted_start_time', 'predicted_step',
       'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3',
       'concept:name - lag_4', 'concept:name - lag_5', 'concept:name - lag_6',
       'concept:name - lag_7', 'concept:name - lag_8', 'concept:name - lag_9',
       'concept:name - lag_10', 'concept:name next'],
      dtype='object')

In [65]:
# Prepare data
X = df_2017[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
y = df_2017['concept:name next']

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
X['concept:name'] = label_encoder.fit_transform(X['concept:name'])
y = label_encoder.fit_transform(y)

# Define the Random Forest Classifier
rf_clf = RandomForestClassifier(n_jobs=-1)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [19, 20, 21],
    'max_depth': [18, 19, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform nested cross-validation
outer_cv = KFold(n_splits=5, shuffle=False, random_state=None)  # 5-fold outer cross-validation
inner_cv = KFold(n_splits=3, shuffle=False, random_state=None)  # 3-fold inner cross-validation

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=inner_cv)
grid_search.fit(X, y)

# Print the results
print(f"Best params: {grid_search.best_params_}")
print(f'Nested Cross-Validation Accuracy: {grid_search.best_score_:.4f}')


/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2043606899.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2043606899.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2043606899.py:9: SettingW

Best params: {'max_depth': 19, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 20}
Nested Cross-Validation Accuracy: 0.9526


In [ ]:
# Predict labels using the best model
df_2017['predicted_next_state'] = best_model.predict(X)